## 라이브러리

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [33]:

import re
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import json
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix, coo_matrix
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import from_networkx
from torch_geometric.nn import SAGEConv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch_geometric.transforms import RandomNodeSplit

## 데이터셋

In [3]:
def generate_graph_data(num_nodes=1000):
    np.random.seed(1127)
    torch.manual_seed(1127)
    
    # 비선형 데이터
    X = np.random.rand(num_nodes, 5)  # 5차원
    y = np.sin(X[:, 0] * 3) + 0.1 * np.random.rand(num_nodes)
    
    edge_index = torch.randint(0, num_nodes, (2, num_nodes * 2))
    
    # PyG 데이터 변환
    return Data(
        x = torch.tensor(X, dtype = torch.float32),
        edge_index= edge_index,
        y = torch.tensor(y, dtype = torch.float32).unsqueeze(1),
    )

def generate_noisy_graph_data(num_nodes=1000, noise_type="gaussian", noise_level=0.1, outlier_ratio=0.05):
    """
    다양한 노이즈를 추가하여 그래프 데이터를 생성하는 함수

    Args:
    - num_nodes (int): 노드 개수
    - noise_type (str): 추가할 노이즈 유형 ("gaussian", "uniform", "outlier", "edge_noise")
    - noise_level (float): 노이즈의 강도 (가우시안 및 유니폼 노이즈)
    - outlier_ratio (float): 이상치(outlier) 비율

    Returns:
    - PyG Data 객체
    """
    np.random.seed(1127)
    torch.manual_seed(1127)
    
    X = np.random.rand(num_nodes, 5)  # 5차원 특징
    y = np.sin(X[:, 0] * 3) + 0.1 * np.random.rand(num_nodes)  # 기본 타겟
    
    if noise_type == "gaussian":
        y += np.random.normal(0, noise_level, size=num_nodes)
    elif noise_type == "uniform":
        y += np.random.uniform(-noise_level, noise_level, size=num_nodes)
    elif noise_type == "outlier":
        num_outliers = int(num_nodes * outlier_ratio)
        outlier_indices = np.random.choice(num_nodes, num_outliers, replace=False)
        y[outlier_indices] += np.random.normal(3, 1.0, size=num_outliers)  # 극단적인 변화

    # 그래프 구조적 노이즈 (엣지 변경)
    edge_index = torch.randint(0, num_nodes, (2, num_nodes * 2))
    if noise_type == "edge_noise":
        # 엣지에 무작위 잡음을 추가하여 구조적 변형 수행
        num_noisy_edges = int(edge_index.shape[1] * noise_level)
        noise_indices = np.random.choice(edge_index.shape[1], num_noisy_edges, replace=False)
        edge_index[:, noise_indices] = torch.randint(0, num_nodes, (2, num_noisy_edges))

    return Data(
        x=torch.tensor(X, dtype=torch.float32),
        edge_index=edge_index,
        y=torch.tensor(y, dtype=torch.float32).unsqueeze(1),
    )
 
def load_graph_data(dataset, category):
    edge_path = f'dataset/{dataset}/{category}/musae_{category}_edges.csv'
    feature_path = f'dataset/{dataset}/{category}/musae_{category}_features.json'
    target_path = f'dataset/{dataset}/{category}/musae_{category}_target.csv'
    
    # 엣지 데이터 로드
    edge_df = pd.read_csv(edge_path)
    edge_index = torch.tensor(edge_df.values.T, dtype=torch.long)
    
    # 피처 데이터 로드
    with open(feature_path, "r") as f:
        features_dict = json.load(f)
    
    node_ids = sorted(map(int, features_dict.keys()))  # 노드 ID 정렬
    node_id_map = {old_id: new_id for new_id, old_id in enumerate(node_ids)}
    
    num_nodes = len(node_ids)
    num_features = max(max(v) for v in features_dict.values()) + 1  # 가장 큰 feature index 찾기
    x = torch.zeros((num_nodes, num_features), dtype=torch.float32)
    
    for node, features in features_dict.items():
        new_id = node_id_map[int(node)]  # 노드 ID 변환
        x[new_id, features] = 1.0  # One-hot 인코딩
    
    # 타겟 데이터 로드
    target_df = pd.read_csv(target_path)
    target_df["id"] = target_df["id"].map(node_id_map)  # 노드 ID 변환
    target_df = target_df.dropna().astype(int)  # 변환되지 않은 노드 제거
    
    # y = torch.zeros(num_nodes, dtype=torch.long)
    y = torch.zeros((num_nodes, 1), dtype=torch.long)  # [, 1] 형태로 변경
    if dataset == 'wiki':
        y[target_df["id"].values] = torch.tensor(target_df["target"].values, dtype=torch.long).view(-1, 1)
    elif dataset == 'twitch':
        y[target_df["id"].values] = torch.tensor(target_df["views"].values, dtype=torch.long).view(-1, 1)
    
    # PyG Data 객체 생성
    graph_data = Data(x=x, edge_index=edge_index, y=y)
    return graph_data

def max_normalize(x):
    return x / np.max(np.abs(x)) if np.max(np.abs(x)) != 0 else x

def std_normalize(x):
    return (x - np.mean(x)) / np.std(x) if np.std(x) != 0 else np.zeros(len(x))

def int_normalize(x):
    return ((x - np.min(x)) / (np.max(x) - np.min(x)) * 2 - 1) if np.std(x) != 0 else np.zeros(len(x))

def simulate_ising(n, h0, J):
    G = nx.grid_2d_graph(n, n)
    l = np.linspace(-1.0, 1.0, n)
    
    s = np.random.choice([-1, 1], size=(n, n))
    # Placeholder for metropolis algorithm
    y = s.flatten()
    f = [[l[i], l[j]] for j in range(n) for i in range(n)]
    
    return G, [nx.to_scipy_sparse_matrix(G)], y, f

def parse_mean_fill(series, normalize=False):
    series = series.replace({',': ''}, regex=True)
    series = pd.to_numeric(series, errors='coerce')
    mean_val = series.mean()
    series.fillna(mean_val, inplace=True)
    
    if normalize:
        series = (series - mean_val) / series.std()
    
    return series.values

def read_county(prediction, year):
    adj = pd.read_csv("dataset/election/adjacency.txt", header=None, sep="\t", dtype=str, encoding="ISO-8859-1")
    fips2cty = {row[1]: row[0] for _, row in adj.iterrows() if pd.notna(row[1])}
    
    hh = adj.iloc[:, 1].ffill().astype(int)
    tt = adj.iloc[:, 3].astype(int)
    
    fips = sorted(set(hh).union(set(tt)))
    id2num = {id_: num for num, id_ in enumerate(fips)}
    
    G = nx.Graph()
    G.add_nodes_from(range(len(id2num)))
    G.add_edges_from([(id2num[h], id2num[t]) for h, t in zip(hh, tt)])
    
    # Load datasets
    VOT = pd.read_csv("dataset/election/election.csv")
    ICM = pd.read_csv("dataset/election/income.csv")
    POP = pd.read_csv("dataset/election/population.csv")
    EDU = pd.read_csv("dataset/election/education.csv")
    UEP = pd.read_csv("dataset/election/unemployment.csv")
    
    cty = pd.DataFrame({'FIPS': fips, 'County': [fips2cty.get(f, '') for f in fips]})
    vot = VOT[['fips_code', f'dem_{year}', f'gop_{year}']].rename(columns={'fips_code': 'FIPS'})
    icm = ICM[['FIPS', f'MedianIncome{min(max(2011, year), 2018)}']]
    pop = POP[['FIPS', f'R_NET_MIG_{min(max(2011, year), 2018)}', f'R_birth_{min(max(2011, year), 2018)}', f'R_death_{min(max(2011, year), 2018)}']]
    edu = EDU[['FIPS', f'BachelorRate{year}']]
    uep = UEP[['FIPS', f'Unemployment_rate_{min(max(2007, year), 2018)}']]
    
    dat = cty.merge(vot, on='FIPS', how='left')
    dat = dat.merge(icm, on='FIPS', how='left')
    dat = dat.merge(pop, on='FIPS', how='left')
    dat = dat.merge(edu, on='FIPS', how='left')
    dat = dat.merge(uep, on='FIPS', how='left')
    
    # Extract features and labels
    dem = parse_mean_fill(dat.iloc[:, 2])
    gop = parse_mean_fill(dat.iloc[:, 3])
    
    ff = np.zeros((len(dat), 7), dtype=np.float32)
    for i in range(6):
        ff[:, i] = parse_mean_fill(dat.iloc[:, i + 4], normalize=True)
    
    ff[:, 6] = (gop - dem) / (gop + dem)
    
    label_mapping = {
        "income": 0, "migration": 1, "birth": 2, "death": 3,
        "education": 4, "unemployment": 5, "election": 6
    }
    
    if prediction not in label_mapping:
        raise ValueError("Unexpected prediction type")
    
    pos = label_mapping[prediction]
    y = ff[:, pos]
    f = [np.concatenate((ff[i, :pos], ff[i, pos + 1:])) for i in range(len(dat))]
    
    return G, [csr_matrix(nx.adjacency_matrix(G))], y, f

def load_county_graph_data(prediction: str, year: int):
    G, A, labels, feats = read_county(prediction, year)

    mapping = {node: i for i, node in enumerate(G.nodes())}
    G = nx.relabel_nodes(G, mapping)

    pyg_data = from_networkx(G)

    edge_index = pyg_data.edge_index
    sorted_edges = torch.sort(edge_index, dim=0)[0]  # (u, v)와 (v, u)를 정렬
    unique_edges = torch.unique(sorted_edges, dim=1)  # 고유 엣지만 유지
    pyg_data.edge_index = unique_edges  # 중복 제거된 edge_index 적용

    pyg_data.x = torch.tensor(feats, dtype=torch.float)

    pyg_data.y = torch.tensor(labels, dtype=torch.float).view(-1, 1)

    return pyg_data

def read_transportation_network(network_name, net_skips, net_cols, netf_cols, flow_skips, flow_cols, V_range):
    # Load data
    dat_net = pd.read_csv(f"dataset/transportation/{network_name}/{network_name}_net.tntp", 
                           skiprows=net_skips, sep='\s+', usecols=net_cols, header=None).values
    dat_netf = pd.read_csv(f"dataset/transportation/{network_name}/{network_name}_net.tntp", 
                            skiprows=net_skips, sep='\s+', usecols=netf_cols, header=None).values
    dat_flow = pd.read_csv(f"dataset/transportation/{network_name}/{network_name}_flow.tntp", 
                            skiprows=flow_skips, sep='\s+', usecols=flow_cols, header=None).values
    
    # Map node labels to indices
    lb2id = {v: i for i, v in enumerate(V_range, start=1)}
    NV = len(V_range)
    
    # Create directed graph
    g = nx.DiGraph()
    g.add_nodes_from(range(1, NV + 1))
    
    for src, dst in dat_net:
        if src in lb2id and dst in lb2id:
            g.add_edge(lb2id[src], lb2id[dst])
    
    # Edge labels
    flow_dict = {}
    for src, dst, flow in dat_flow:
        if src in lb2id and dst in lb2id:
            flow_dict[(lb2id[src], lb2id[dst])] = flow
    
    y = np.array([flow_dict.get((e[0], e[1]), 0) for e in g.edges()])
    y = (y - np.mean(y)) / np.std(y)  # Standard normalization
    
    # Edge features
    netf_dict = {}
    for i in range(len(dat_net)):
        src, dst = dat_net[i]
        if src in lb2id and dst in lb2id:
            netf_dict[(lb2id[src], lb2id[dst])] = dat_netf[i]
    
    ff = np.array([netf_dict[e] for e in g.edges()])
    mean_ff = np.mean(ff, axis=0)
    std_ff = np.std(ff, axis=0)
    std_ff[std_ff == 0] = 1  # Prevent division by zero
    netf = (ff - mean_ff) / std_ff  
    
    f = list(netf)
    
    # Line graph transformation
    G1 = nx.Graph()
    G2 = nx.Graph()
    sorted_edges = sorted(g.edges())
    tuple2id = {e: i for i, e in enumerate(sorted_edges)}
    
    for u in g.nodes:
        innbrs = list(g.predecessors(u))
        outnbrs = list(g.successors(u))
        
        for v in innbrs:
            for w in outnbrs:
                if (v, u) in tuple2id and (u, w) in tuple2id:
                    G1.add_edge(tuple2id[(v, u)], tuple2id[(u, w)])
        
        for v in innbrs:
            for w in innbrs:
                if w > v and (v, u) in tuple2id and (w, u) in tuple2id:
                    G2.add_edge(tuple2id[(v, u)], tuple2id[(w, u)])
        
        for v in outnbrs:
            for w in outnbrs:
                if w > v and (u, v) in tuple2id and (u, w) in tuple2id:
                    G2.add_edge(tuple2id[(u, v)], tuple2id[(u, w)])
                    
    size = max(len(G1.nodes), len(G2.nodes))
    A1 = np.zeros((size, size))
    A2 = np.zeros((size, size))
    
    A1[:nx.number_of_nodes(G1), :nx.number_of_nodes(G1)] = nx.adjacency_matrix(G1).todense()
    A2[:nx.number_of_nodes(G2), :nx.number_of_nodes(G2)] = nx.adjacency_matrix(G2).todense()
    
    A = A1 + A2
    
    return nx.Graph(A), A, y, f

def load_trans_graph_data(city: str):
    if city == 'Anaheim':
        G, A, labels, feats = read_transportation_network(city, 8, [0, 1], [2, 3, 4, 7], 6, [0, 1, 3], range(1, 417))
    elif city == 'ChicagoSketch':
        G, A, labels, feats = read_transportation_network(city, 7, [0, 1], [2, 3, 4, 7], 1, [0, 1, 2], range(388, 934))

    mapping = {node: i for i, node in enumerate(G.nodes())}
    G = nx.relabel_nodes(G, mapping)

    pyg_data = from_networkx(G)

    edge_index = pyg_data.edge_index
    sorted_edges = torch.sort(edge_index, dim=0)[0]  # (u, v)와 (v, u)를 정렬
    unique_edges = torch.unique(sorted_edges, dim=1)  # 고유 엣지만 유지
    pyg_data.edge_index = unique_edges  # 중복 제거된 edge_index 적용

    pyg_data.x = torch.tensor(feats, dtype=torch.float)

    pyg_data.y = torch.tensor(labels, dtype=torch.float).view(-1, 1)

    return pyg_data

In [4]:
# 기본 비선형 그래프 데이터
graph_data_basic = generate_graph_data(num_nodes=1000)

# 노이즈 비선형 그래프 데이터
graph_data_noise_gaussian = generate_noisy_graph_data(num_nodes=1000, noise_type='gaussian', noise_level=0.2)
graph_data_noise_uniform = generate_noisy_graph_data(num_nodes=1000, noise_type='uniform', noise_level=0.2)
graph_data_noise_outlier = generate_noisy_graph_data(num_nodes=1000, noise_type='outlier', noise_level=0.2)
graph_data_noise_edge = generate_noisy_graph_data(num_nodes=1000, noise_type='edge_noise', noise_level=0.2)

# County 데이터셋
graph_data_county_edu = load_county_graph_data('education', 2012)
graph_data_county_elec = load_county_graph_data('election', 2012)
graph_data_county_inc = load_county_graph_data('income', 2012)
graph_data_county_unemp = load_county_graph_data('unemployment', 2012)

# Twitch 데이터셋
graph_data_twitch_de = load_graph_data('twitch', 'DE')
graph_data_twitch_engb = load_graph_data('twitch', 'ENGB')
graph_data_twitch_es = load_graph_data('twitch', 'ES')
graph_data_twitch_fr = load_graph_data('twitch', 'FR')
graph_data_twitch_ptbr = load_graph_data('twitch', 'PTBR')
graph_data_twitch_ru = load_graph_data('twitch', 'RU')

# Wikipedia 데이터
graph_data_wiki_ch = load_graph_data('wikipedia', 'chameleon')
graph_data_wiki_cr = load_graph_data('wikipedia', 'crocodile')
graph_data_wiki_sq = load_graph_data('wikipedia', 'squirrel')

# Transfortation 데이터셋
graph_data_trans_ana = load_trans_graph_data('Anaheim')
graph_data_trans_chica = load_trans_graph_data('ChicagoSketch')

/tmp/ipykernel_419030/223594441.py:202: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  pyg_data.x = torch.tensor(feats, dtype=torch.float)


In [5]:
# 그래프 데이터 리스트 
graph_datasets = {
    "Basic": graph_data_basic,
    "Noise_Gaussian": graph_data_noise_gaussian,
    "Noise_Uniform": graph_data_noise_uniform,
    "Noise_Outlier": graph_data_noise_outlier,
    "Noise_Edge": graph_data_noise_edge,
    "County_Education": graph_data_county_edu,
    "County_Election": graph_data_county_elec,
    "County_Income": graph_data_county_inc,
    "County_Unemployment": graph_data_county_unemp,
    "Twitch_DE": graph_data_twitch_de,
    "Twitch_ENGB": graph_data_twitch_engb,
    "Twitch_ES": graph_data_twitch_es,
    "Twitch_FR": graph_data_twitch_fr,
    "Twitch_PTBR": graph_data_twitch_ptbr,
    "Twitch_RU": graph_data_twitch_ru,
    "Wikipedia_Chameleon": graph_data_wiki_ch,
    "Wikipedia_Crocodile": graph_data_wiki_cr,
    "Wikipedia_Squirrel": graph_data_wiki_sq,
    "Transportation_Anaheim": graph_data_trans_ana,
    "Transportation_Chicago": graph_data_trans_chica,
}

# 데이터 저장 리스트
graph_info_list = []

for dataset_name, graph_data in graph_datasets.items():
    if graph_data is None:
        continue  # 그래프 데이터가 없으면 스킵
    
    nodes = graph_data.x.shape[0] if hasattr(graph_data, "x") else 0  # 노드 개수
    edges = graph_data.edge_index.shape[1] if hasattr(graph_data, "edge_index") else 0  # 엣지 개수
    features = graph_data.x.shape[1] if hasattr(graph_data, "x") else 0  # 피처 개수
    
    graph_info_list.append({"Dataset": dataset_name, "Nodes": nodes, "Edges": edges, "Features": features})

# 데이터 프레임 생성
df_graph_info = pd.DataFrame(graph_info_list)
df_graph_info

,Dataset,Nodes,Edges,Features
0,Basic,1000,2000,5
1,Noise_Gaussian,1000,2000,5
2,Noise_Uniform,1000,2000,5
3,Noise_Outlier,1000,2000,5
4,Noise_Edge,1000,2000,5
5,County_Education,3234,12717,6
6,County_Election,3234,12717,6
7,County_Income,3234,12717,6
8,County_Unemployment,3234,12717,6
9,Twitch_DE,9498,153138,3170


## 모델 학습

In [44]:
def split_graph_data(graph_data, test_ratio=0.2):
    transform = RandomNodeSplit(split="train_rest", num_val=0.0, num_test=test_ratio)
    data = transform(graph_data)

    # Train/Test 마스크 생성
    train_mask = data.train_mask.to(data.x.device)
    test_mask = data.test_mask.to(data.x.device)

    # 훈련 및 테스트 데이터 선택
    train_data = data.clone()
    test_data = data.clone()

    train_data.x = data.x[train_mask]
    train_data.y = data.y[train_mask]
    test_data.x = data.x[test_mask]
    test_data.y = data.y[test_mask]

    # 노드 인덱스를 매핑하여 edge_index 수정 (훈련 데이터)
    train_node_mapping = {old_idx.item(): new_idx for new_idx, old_idx in enumerate(train_mask.nonzero(as_tuple=True)[0])}
    train_edges_mask = train_mask[data.edge_index[0]] & train_mask[data.edge_index[1]]
    train_edges = data.edge_index[:, train_edges_mask]

    train_data.edge_index = torch.stack([
        torch.tensor([train_node_mapping[i.item()] for i in train_edges[0]], dtype=torch.long, device=data.x.device),
        torch.tensor([train_node_mapping[i.item()] for i in train_edges[1]], dtype=torch.long, device=data.x.device)
    ], dim=0)

    # 노드 인덱스를 매핑하여 edge_index 수정 (테스트 데이터)
    test_node_mapping = {old_idx.item(): new_idx for new_idx, old_idx in enumerate(test_mask.nonzero(as_tuple=True)[0])}
    test_edges_mask = test_mask[data.edge_index[0]] & test_mask[data.edge_index[1]]
    test_edges = data.edge_index[:, test_edges_mask]

    test_data.edge_index = torch.stack([
        torch.tensor([test_node_mapping[i.item()] for i in test_edges[0]], dtype=torch.long, device=data.x.device),
        torch.tensor([test_node_mapping[i.item()] for i in test_edges[1]], dtype=torch.long, device=data.x.device)
    ], dim=0)

    print(f"Train data: {train_data.x.shape[0]} nodes, {train_data.edge_index.shape[1]} edges")
    print(f"Test data: {test_data.x.shape[0]} nodes, {test_data.edge_index.shape[1]} edges")
    
    return train_data, test_data

def augment_features(x, tau):
    tau = tau.view(-1, 1)
    tau_transformed = (tau - 0.5) * 12 # 분위수 값 변환: 학습 안정성 증가
    
    return torch.cat((x, tau_transformed.expand(x.size(0), -1)), dim = 1)

def coverage_loss(y_true, y_low, y_upper, target = 0.9):
    coverage_exp = ((y_true >= y_low) & (y_true <= y_upper)).float().mean()
    
    return (coverage_exp - target) ** 2

def dynamic_quantile_adjustment(tau_low, tau_upper, coverage, target=0.9, learning_rate=0.005, min_gap=0.05):
    coverage_error = coverage - target  
    tau_adjustment = max(0.001, min(0.05, learning_rate * abs(coverage_error)))  # 과도한 조정 방지

    # 분위수 업데이트
    tau_low_new = max(0.01, tau_low - tau_adjustment if coverage > target else tau_low + tau_adjustment)
    tau_upper_new = min(0.99, tau_upper + tau_adjustment if coverage > target else tau_upper - tau_adjustment)

    # 최소 간격 유지
    if tau_upper_new - tau_low_new < min_gap:
        tau_mid = (tau_low_new + tau_upper_new) / 2
        tau_low_new = max(0.01, tau_mid - min_gap / 2)
        tau_upper_new = min(0.99, tau_mid + min_gap / 2)

    return tau_low_new, tau_upper_new

class GQNN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels + 1, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.fc = nn.Linear(hidden_channels, 1)
        
    def forward(self, x, edge_index, tau):
        x = augment_features(x, tau)
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        
        return self.fc(x)

class QRLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, y_pred, y_true, tau):
        diff = y_true - y_pred
        loss = torch.where(diff > 0, tau * diff, (tau - 1) * diff)
        
        return torch.mean(loss) 

class GQNN_PI(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.fc = nn.Linear(hidden_channels, 2)
        
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.fc(x)
        
        # 분위수 교차 방지 (q1 ≤ q2 보장)
        q1, q2 = torch.unbind(x, dim=1)  # (num_nodes, 2) → q1, q2로 분리
        q1, q2 = torch.min(q1, q2), torch.max(q1, q2)  # 분위수 순서 보장
        
        return torch.stack([q1, q2], dim=1)

class RQRLoss(nn.Module):
    def __init__(self, target_coverage=0.9, lambda_factor=0.1):
        super(RQRLoss, self).__init__()
        self.target_coverage = target_coverage
        self.lambda_factor = lambda_factor

    def forward(self, q1, q2, target):
        width = torch.relu(q2 - q1)  # 신뢰구간 너비가 음수가 되지 않도록 방지

        # 분위수 회귀 손실
        diff1 = target - q1
        diff2 = target - q2
        quantile_loss = torch.maximum(diff1 * diff2, torch.tensor(0.0, device=target.device))

        # 신뢰구간 너비 최소화 정규화
        width_penalty = self.lambda_factor * width ** 2 * 0.5

        # Coverage Loss 추가
        coverage = ((target >= q1) & (target <= q2)).float().mean()
        coverage_loss = (coverage - self.target_coverage) ** 2

        return torch.mean(quantile_loss + width_penalty + coverage_loss)

def train_fixed_gqnn(train_data, q='all', hidden_channels=64, num_epochs=1000, learning_rate=0.001, weight=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    in_channels = train_data.x.shape[1]
    model = GQNN(in_channels=in_channels, hidden_channels=hidden_channels).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight)
    criterion = QRLoss()
    train_data = train_data.to(device)

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        
        if q == "all":
            taus = torch.rand(train_data.x.size(0), 1, dtype=torch.float32, device=device)
        else:
            taus = torch.full((train_data.x.size(0), 1), q, dtype=torch.float32, device=device)

        preds = model(train_data.x, train_data.edge_index, taus)

        loss = criterion(preds, train_data.y, taus)

        loss.backward()
        optimizer.step()

        if epoch % (num_epochs // 10) == 0:
            print(f"Epoch {epoch}: Loss = {loss.item():.4f}")

    return model

def train_dynamic_gqnn(train_data, q='all', tau_low=0.05, tau_upper=0.95, hidden_channels=64, num_epochs=1000, learning_rate=0.001, weight=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    in_channels = train_data.x.shape[1]
    model = GQNN(in_channels=in_channels, hidden_channels=hidden_channels).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight)
    criterion = QRLoss()
    train_data = train_data.to(device)

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        
        if q == "all":
            taus_low = (torch.rand(train_data.x.size(0), 1) * (0.5 - tau_low) + tau_low).to(device)
            taus_upper = (torch.rand(train_data.x.size(0), 1) * (tau_upper - 0.5) + 0.5).to(device)
        elif q == 'single':
            taus_low = torch.full((train_data.x.size(0), 1), tau_low, device=device)
            taus_upper = torch.full((train_data.x.size(0), 1), tau_upper, device=device)

        preds_low = model(train_data.x, train_data.edge_index, taus_low)
        preds_upper = model(train_data.x, train_data.edge_index, taus_upper)
        
        loss_low = criterion(preds_low, train_data.y, taus_low)
        loss_upper = criterion(preds_upper, train_data.y, taus_upper)
        loss = loss_low + loss_upper
        
        loss.backward()
        optimizer.step()

        coverage = ((train_data.y >= preds_low) & (train_data.y <= preds_upper)).float().mean().item()
        target_coverage = 0.9
        tau_low, tau_upper = dynamic_quantile_adjustment(tau_low, tau_upper, coverage, target=target_coverage)
        
        if epoch % (num_epochs // 10) == 0:
            print(f"Epoch {epoch}: Loss = {loss.item():.4f}, Coverage Error: {target_coverage - coverage:.4f}")

    return model, tau_low, tau_upper

def train_pi_gqnn(train_data, hidden_channels=64, num_epochs=1000, learning_rate=0.001, weight=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    in_channels = train_data.x.shape[1]
    model = GQNN_PI(in_channels=in_channels, hidden_channels=hidden_channels).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight)
    criterion = RQRLoss()
    train_data = train_data.to(device)

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        
        preds = model(train_data.x, train_data.edge_index)
        loss = criterion(preds[:, 0], preds[:, 1], train_data.y)
        
        loss.backward()
        optimizer.step()

        if epoch % (num_epochs // 10) == 0:
            print(f"Epoch {epoch}: Loss = {loss.item():.4f}")

    return model


In [ ]:
train_data_bs, test_data_bs = split_graph_data(graph_data_basic)

model_fixed_all = train_fixed_gqnn(train_data_bs, q='all')
model_fixed_05 = train_fixed_gqnn(train_data_bs, q=0.05)
model_fixed_95 = train_fixed_gqnn(train_data_bs, q=0.95)

model_dynamic_all = train_dynamic_gqnn(train_data_bs, q='all')
model_dynamic_single = train_dynamic_gqnn(train_data_bs, q='single')

model_pi = train_pi_gqnn(train_data_bs)

Train data: 800 nodes, 1279 edges
Test data: 200 nodes, 84 edges
Epoch 0: Loss = 0.3740
Epoch 100: Loss = 0.0848
Epoch 200: Loss = 0.0786


In [35]:
from torch_geometric.transforms import RandomNodeSplit
import torch

# 데이터 분할 (RandomNodeSplit 사용)
transform = RandomNodeSplit(split="train_rest", num_val=0.0, num_test=0.2)
data = transform(graph_data_basic)

# Train/Test 마스크 생성
train_mask = data.train_mask.to(data.x.device)
test_mask = data.test_mask.to(data.x.device)

# 훈련 및 테스트 데이터 선택
train_data = data.clone()
test_data = data.clone()

train_data.x = data.x[train_mask]
train_data.y = data.y[train_mask]
test_data.x = data.x[test_mask]
test_data.y = data.y[test_mask]

# 노드 인덱스를 매핑하여 edge_index 수정 (훈련 데이터)
train_node_mapping = {old_idx.item(): new_idx for new_idx, old_idx in enumerate(train_mask.nonzero(as_tuple=True)[0])}
train_edges_mask = train_mask[data.edge_index[0]] & train_mask[data.edge_index[1]]
train_edges = data.edge_index[:, train_edges_mask]

train_data.edge_index = torch.stack([
    torch.tensor([train_node_mapping[i.item()] for i in train_edges[0]], dtype=torch.long, device=data.x.device),
    torch.tensor([train_node_mapping[i.item()] for i in train_edges[1]], dtype=torch.long, device=data.x.device)
], dim=0)

# 노드 인덱스를 매핑하여 edge_index 수정 (테스트 데이터)
test_node_mapping = {old_idx.item(): new_idx for new_idx, old_idx in enumerate(test_mask.nonzero(as_tuple=True)[0])}
test_edges_mask = test_mask[data.edge_index[0]] & test_mask[data.edge_index[1]]
test_edges = data.edge_index[:, test_edges_mask]

test_data.edge_index = torch.stack([
    torch.tensor([test_node_mapping[i.item()] for i in test_edges[0]], dtype=torch.long, device=data.x.device),
    torch.tensor([test_node_mapping[i.item()] for i in test_edges[1]], dtype=torch.long, device=data.x.device)
], dim=0)

# 결과 출력
print(f"Train data: {train_data.x.shape[0]} nodes, {train_data.edge_index.shape[1]} edges")
print(f"Test data: {test_data.x.shape[0]} nodes, {test_data.edge_index.shape[1]} edges")

Train data: 800 nodes, 1259 edges
Test data: 200 nodes, 78 edges


In [40]:
test_data.x.shape

torch.Size([200, 5])

In [ ]:
# 주변 노드 정보를 고려하여 신뢰 구간을 학습하는 GNN
# 고정된 분위수가 아닌 모델이 학습하면서 신뢰구간을 예측하도록 설계 -> 유연한 방식
class QuantileGNNLayer(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(QuantileGNNLayer, self).__init__()
        self.conv = SAGEConv(in_channels, hidden_channels)  # 그래프 메세지 패싱을 수행하는 레이어
        self.fc = nn.Linear(hidden_channels, out_channels)  # 최종 출력을 위한 선형 변환

    def forward(self, x, edge_index):
        h = self.conv(x, edge_index)  # GNN 메세지 패싱 수행
        h = F.relu(h)   # 활성화 함수 적용 (비선형 변환)
        h = self.fc(h)  # 최종 분위수 얘측
        return h  # (num_nodes, 2) 반환 (ql qu)

# 신뢰 구간을 예측하는 GNN 모델
class QuantileGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=2):
        super(QuantileGNN, self).__init__()
        self.layer1 = QuantileGNNLayer(input_dim, hidden_dim, hidden_dim)
        self.layer2 = QuantileGNNLayer(hidden_dim, hidden_dim, output_dim)  # 최종 출력 2차원 (q1, q2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.layer1(x, edge_index))
        x = self.layer2(x, edge_index)  # 최종 분위수 출력
        return x  # (num_nodes, 2) 반환

class DynamicRQRLoss(nn.Module):
    def __init__(self, target_coverage=0.9, lambda_factor=0.1):
        super(DynamicRQRLoss, self).__init__()
        self.target_coverage = target_coverage
        self.lambda_factor = lambda_factor

    def forward(self, preds, target):
        q1, q2 = preds[:, 0], preds[:, 1]
        width = q2 - q1

        # 기존 RQRW Loss (완화된 분위수 회귀)
        diff_mu_1 = target - q1
        diff_mu_2 = target - q2
        rqr_loss = torch.maximum(diff_mu_1 * diff_mu_2 * (self.target_coverage + 2 * self.lambda_factor),
                                 diff_mu_2 * diff_mu_1 * (self.target_coverage + 2 * self.lambda_factor - 1))

        # 신뢰구간 너비 최소화 정규화
        width_penalty = self.lambda_factor * torch.square(width) * 0.5
        
        # 분위수 교차 방지?

        return torch.mean(rqr_loss + width_penalty)

def train_quantile_gnn(graph_data, train_data, epochs=1000, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 모델 초기화
    input_dim = graph_data.x.shape[1]
    model = QuantileGNN(input_dim=input_dim, hidden_dim=64).to(device)
    criterion = DynamicRQRLoss(target_coverage=0.9, lambda_factor=0.1)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_mask = torch.tensor([i in train_data for i in range(graph_data.x.shape[0])])
    
    graph_data = graph_data.to(device)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        preds = model(graph_data)
        loss = criterion(preds[train_mask], graph_data.y[train_mask])
        loss.backward()
        optimizer.step() # 모델 업데이트

        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Loss = {loss.item():.4f}")

    return model

def test_quantile_gnn(model, graph_data, test_data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()  # 모델을 평가 모드로 설정

    test_mask = torch.tensor([i in test_data for i in range(graph_data.x.shape[0])])
    graph_data = graph_data.to(device)

    with torch.no_grad():  # 그래디언트 계산 없이 예측 수행
        test_preds = model(graph_data)[test_mask]  # (num_test_nodes, 2) → 분위수 예측값
        test_targets = graph_data.y[test_mask]  # 실제 정답

    return test_preds.cpu().numpy(), test_targets.cpu().numpy()

def evaluate_model_performance(test_preds, test_targets, coverage_target=0.9):
    ql = test_preds[:, 0]  # 신뢰구간 하한 (예측값)
    qu = test_preds[:, 1]  # 신뢰구간 상한 (예측값)
    
    coverage = np.mean((test_targets >= ql) & (test_targets <= qu))

    interval_width = np.mean(qu - ql)
    
    mae = np.mean(np.abs((ql + qu) / 2 - test_targets))  # 신뢰구간 중앙값과 실제 값 비교

    print(f"   - Coverage Rate: {coverage:.4f} (목표: {coverage_target})")
    print(f"   - Prediction Interval Width: {interval_width:.4f}")
    print(f"   - Mean Absolute Error (MAE): {mae:.4f}\n")

    # return coverage, interval_width, mae

def advanced_evaluate_model(test_preds, test_targets, coverage_target=0.9):
    """
    추가적인 평가 지표를 포함한 모델 평가 함수
    """
    q1 = test_preds[:, 0]  # 신뢰구간 하한
    q2 = test_preds[:, 1]  # 신뢰구간 상한
    median_pred = (q1 + q2) / 2  # 신뢰구간 중앙값

    coverage = np.mean((test_targets >= q1) & (test_targets <= q2))

    interval_width = np.mean(q2 - q1)

    mae = np.mean(np.abs(median_pred - test_targets))

    sharpness = np.mean(np.square(q2 - q1))

    alpha = 1 - coverage_target  # 신뢰구간 목표
    lower_penalty = np.maximum(q1 - test_targets, 0) * (alpha / 2)
    upper_penalty = np.maximum(test_targets - q2, 0) * (alpha / 2)
    interval_score = interval_width + 2 * (lower_penalty + upper_penalty).mean()

    def pinball_loss(y_true, y_pred, quantile):
        return np.maximum(quantile * (y_true - y_pred), (quantile - 1) * (y_true - y_pred)).mean()
    
    pinball_loss_lower = pinball_loss(test_targets, q1, alpha / 2)
    pinball_loss_upper = pinball_loss(test_targets, q2, 1 - alpha / 2)

    calibration_error = np.abs(coverage - coverage_target)

    print(f"   - Coverage Rate: {coverage:.4f} (목표: {coverage_target})")
    print(f"   - Prediction Interval Width (PIW): {interval_width:.4f}")
    print(f"   - Mean Absolute Error (MAE): {mae:.4f}")
    print(f"   - Sharpness: {sharpness:.4f} (작을수록 좋음)")
    print(f"   - Interval Score (IS): {interval_score:.4f} (작을수록 좋음)")
    print(f"   - Pinball Loss (Lower): {pinball_loss_lower:.4f}, (Upper): {pinball_loss_upper:.4f}")
    print(f"   - Calibration Error: {calibration_error:.4f} (0에 가까울수록 좋음)\n")

    return {
        "coverage": coverage,
        "interval_width": interval_width,
        "mae": mae,
        "sharpness": sharpness,
        "interval_score": interval_score,
        "pinball_loss_lower": pinball_loss_lower,
        "pinball_loss_upper": pinball_loss_upper,
        "calibration_error": calibration_error
    }

def visualize_results(preds, targets, title="Quantile GNN - Prediction Intervals"):
    lower_bound = preds[:, 0]  # 신뢰구간 하한
    upper_bound = preds[:, 1]  # 신뢰구간 상한

    plt.figure(figsize=(15, 5))
    plt.plot(targets, label="True Values", marker='o', alpha = 0.5)
    plt.fill_between(range(len(targets)), lower_bound, upper_bound, color='blue', alpha=0.3, label="Prediction Intervals(target coverage = 90%)")
    plt.legend()
    plt.xlabel("Node Index")
    plt.ylabel("Prediction")
    plt.title(title)
    plt.show()
    
def experiment_results_to_df(experiment_results):
    data = []
    for noise_type, results in experiment_results.items():
        train_metrics = results["train"]
        test_metrics = results["test"]
        
        data.append([
            noise_type.upper(),  
            train_metrics["coverage"], test_metrics["coverage"],  
            train_metrics["interval_width"], test_metrics["interval_width"],
            train_metrics["mae"], test_metrics["mae"],
            train_metrics["interval_score"], test_metrics["interval_score"],
        ])
    
    df = pd.DataFrame(data, columns=[
        "Noise Type",  
        "Train Coverage", "Test Coverage",  
        "Train Interval Width", "Test Interval Width",
        "Train MAE", "Test MAE",
        "Train Interval Score", "Test Interval Score"
    ])
    
    return df

def plot_experiment_results(experiment_df):
    noise_types = experiment_df["Noise Type"]
    
    metrics = [
        ("Train Coverage", "Test Coverage"),
        ("Train Interval Width", "Test Interval Width"),
        ("Train MAE", "Test MAE"),
        ("Train Interval Score", "Test Interval Score")
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    axes = axes.flatten()
    
    for i, (train_metric, test_metric) in enumerate(metrics):
        train_values = experiment_df[train_metric]
        test_values = experiment_df[test_metric]
        
        x = np.arange(len(noise_types))
        width = 0.35
        
        axes[i].bar(x - width/2, train_values, width, label="Train")
        axes[i].bar(x + width/2, test_values, width, label="Test")
        
        axes[i].set_xticks(x)
        axes[i].set_xticklabels(noise_types, rotation=45)
        axes[i].set_title(f"{train_metric} vs {test_metric}")
        axes[i].legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 모델 학습
train_data_bs, test_data_bs = train_test_split(range(graph_data_bs.x.shape[0]), test_size=0.2, random_state=127)
model_bs = train_quantile_gnn(graph_data_bs, train_data_bs, epochs=1000, learning_rate=0.001)

# 테스트
train_preds_bs, train_targets_bs = test_quantile_gnn(model_bs, graph_data_bs, train_data_bs)
test_preds_bs, test_targets_bs = test_quantile_gnn(model_bs, graph_data_bs, test_data_bs)

In [ ]:
print('train evaluation')
advanced_evaluate_model(train_preds_bs, train_targets_bs)
print('-' * 50)
print('test evaluation')
advanced_evaluate_model(test_preds_bs, test_targets_bs)
print('-' * 50)
print('Visualization')
visualize_results(train_preds_bs, train_targets_bs, title="Train Data - Prediction Intervals")
visualize_results(test_preds_bs, test_targets_bs, title="Test Data - Prediction Intervals")